In [1]:
!git clone https://github.com/alycialee/beyond-scale-language-data-diversity.git
%cd beyond-scale-language-data-diversity

# 2️⃣  Make sure the build tools are modern enough
# %pip install --quiet --upgrade pip setuptools wheel

%pip install pip==24.0

# 3️⃣  Editable-install *into the live kernel*  ← note the %pip magic
%pip install -e .

!pip install -U datasets
!pip install fastdtw

fatal: destination path 'beyond-scale-language-data-diversity' already exists and is not an empty directory.
/content/beyond-scale-language-data-diversity
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Obtaining file:///content/beyond-scale-language-data-diversity
  Preparing metadata (setup.py) ... done
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Att

In [2]:
import sys, pathlib
sys.path.insert(0, str(pathlib.Path.cwd() / "src"))
sys.path.insert(0, str(pathlib.Path.cwd() / "src" / "diversity"))

In [3]:
def make_loss_fn(ignore_id):
    """Factory function to create a cross-entropy loss function."""
    def _loss_fn(logits, tgt, *_, **__):
        logits = logits[:, :-1, :].contiguous()
        tgt = tgt[:, 1:].contiguous()
        return F.cross_entropy(logits.view(-1, logits.size(-1)), tgt.view(-1), ignore_index=ignore_id)
    return _loss_fn

In [4]:
import os
# ---------------------------------------------------------------------------
# Helper: one call = one embedding/LM-loss for a given solver tokenizer
# ---------------------------------------------------------------------------
def embed_with_solver_tokenizer(
    solver_model_id: str,
    raw_dataset,
    probe_model,
    max_len: int,
    epochs: int = 1,
    cpu_workers: int = os.cpu_count(),
):
    """
    Parameters
    ----------
    solver_model_id : str
        Hugging Face model ID from which to load the tokenizer.
    raw_dataset : datasets.Dataset
        The *raw_ds* already built earlier in the script.
    probe_model : transformers.PreTrainedModel
        The already-constructed probe (distilgpt2 here).
    max_len : int
        Sequence length to which examples are padded/truncated
        (probe_model.config.max_position_embeddings).
    cpu_workers : int, optional
        How many worker processes to give 🤗 Datasets for .map()

    Returns
    -------
    embedding_tensor : torch.FloatTensor  (shape = probe hidden_dim × probe hidden_dim)
    lm_loss          : float | None
    """

    # ----- Tokenizer --------------------------------------------------------
    tok = AutoTokenizer.from_pretrained(solver_model_id, trust_remote_code=True)
    if tok.pad_token is None:                       # ensure a pad token exists
        tok.add_special_tokens({'pad_token': '[PAD]'})

    probe_model.resize_token_embeddings(len(tok))   # probe needs same vocab size

    # ----- Tokenize dataset -------------------------------------------------
    tok_ds = raw_dataset.map(
        lambda b: tok(
            b["text"],
            padding="longest",
            truncation=True,
            max_length=max_len,
        ),
        batched=True,
        batch_size=20,
        remove_columns=raw_dataset.column_names,
        num_proc=cpu_workers,
    )
    tok_ds.set_format(type="torch", columns=["input_ids", "attention_mask"])

    # ----- Task2Vec ---------------------------------------------------------
    task2vec = Task2Vec(
        probe_model,
        max_samples=1024,
        loader_opts={"batch_size": 1, "shuffle": True, "num_workers": 0},
    )
    task2vec.loss_fn = make_loss_fn(tok.pad_token_id)

    emb, lm_loss = task2vec.embed(tok_ds, epochs=epochs)

    # Convert to torch tensor for downstream use
    embedding_tensor = torch.from_numpy(emb.hessian).to(dtype=torch.float32)

    # Explicitly free large objects to keep memory usage down
    del tok, tok_ds, task2vec, emb
    gc.collect()
    torch.cuda.empty_cache()  # uncomment if you are using CUDA

    return embedding_tensor, lm_loss


In [15]:
pip install --upgrade huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.3/515.3 kB 12.8 MB/s eta 0:00:00
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.33.2
    Uninstalling huggingface-hub-0.33.2:
      Successfully uninstalled huggingface-hub-0.33.2


In [ ]:
!echo -e "HF_TOKEN\ny" | huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
/usr/lib/python3.11/getpass.py:91: GetPassWarning: Can not control echo on the terminal.
  passwd = fallback_getpass(prompt, stream)
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Token is valid (permission: read).
The token `a` has been saved to /root/.cache/huggingface/stored_tokens


In [19]:
import os
import gc
import torch
import warnings
import torch.nn.functional as F
from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoConfig,
)
from task2vec import Task2Vec
from itertools import combinations
from torch.utils.data import DataLoader
from functools import lru_cache
from typing import Callable, Tuple
from torch import Tensor
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean



warnings.filterwarnings("ignore", module="torch")

# ----- Global Configuration & Models -----------------------------------------
print(">>> Loading and preparing dataset...")
raw_ds = load_dataset("Tonic/MiniF2F", split="train")
raw_ds = raw_ds.select(range(5)) # Using a smaller subset for demonstration
raw_ds = raw_ds.map(
    lambda ex: {"text": "\n".join(p for p in (ex["informal_prefix"], ex["formal_statement"], ex["goal"]) if p)},
    num_proc=os.cpu_count(),
)
print(">>> Dataset ready.")


# Define the device to use (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"--- Using device: {device} ---")

print(">>> Loading probe model...")
# probe_model_id = "distilbert/distilgpt2"
probe_model_id = "Saisam/gpt-neo-math-small"
probe_cfg = AutoConfig.from_pretrained(probe_model_id, trust_remote_code=True)
probe_cfg.attn_implementation = "sdpa"
probe_model = AutoModelForCausalLM.from_pretrained(
    probe_model_id,
    config=probe_cfg,
    trust_remote_code=True,
)

# Move the model to the defined device (GPU)
probe_model.to(device)

probe_model.gradient_checkpointing_enable()
max_probe_length = probe_model.config.max_position_embeddings
print(f">>> Probe model '{probe_model_id}' loaded with max length: {max_probe_length}.")

# ----- Helper & Core Functions -----------------------------------------------

@lru_cache(maxsize=128)
def compute_cmdiv(
    model_id1: str,
    model_id2: str,
    dataset_fingerprint: str,
    batch_size: int
) -> float:
    """
    Computes cmdiv by averaging embedding distances over batches.
    The `align_fn` is called on each pair of flattened embeddings
    and should return the two tensors to compare.
    """
    print(f"\n>>> Computing cmdiv for pair: ({model_id1}, {model_id2})")
    loader = DataLoader(raw_ds, batch_size=batch_size)
    distances = []

    for i, batch in enumerate(loader, start=1):
        print(f"    ... processing batch {i}/{len(loader)}")
        batch_ds = Dataset.from_dict(batch)

        emb1, _ = embed_with_solver_tokenizer(
            model_id1, batch_ds, probe_model, max_probe_length
        )
        emb2, _ = embed_with_solver_tokenizer(
            model_id2, batch_ds, probe_model, max_probe_length
        )

        a = emb1.flatten()
        b = emb2.flatten()

        dtw_dist, path = fastdtw(
            a.numpy(),
            b.numpy(),
            dist=lambda x, y: abs(x - y)
        )

        # 3) normalize by path length to get per-step average
        avg_diff = dtw_dist / len(path)

        # 4) turn into a [0,1] similarity if you like
        sim_dtw = 1.0 / (1.0 + avg_diff)

        print(f"dist between {model_id1} and {model_id2}: {sim_dtw}")
        # distance
        dist = 1.0 - sim_dtw
        distances.append(dist)

    avg_distance = sum(distances) / len(distances) if distances else 0.0
    print(f">>> Average distance (cmdiv) for pair = {avg_distance:.4f}")
    return avg_distance

def compute_edc(model_ids: set, dataset: Dataset, batch_size: int) -> float:
    """
    Computes the Ensemble Diversity Coefficient (EDC) for a set of models.
    """
    if len(model_ids) < 2:
        return 0.0

    dataset_fingerprint = dataset.info.builder_name
    pairwise_diversities = []

    for m_id1, m_id2 in combinations(sorted(list(model_ids)), 2):
        diversity = compute_cmdiv(m_id1, m_id2, dataset_fingerprint, batch_size)
        print(f"Diversity between {m_id1} and {m_id2} on {dataset_fingerprint} with batch size {batch_size}: {diversity}")
        pairwise_diversities.append(diversity)

    return sum(pairwise_diversities) / len(pairwise_diversities) if pairwise_diversities else 0.0

# ----- Main Execution --------------------------------------------------------

solver_model_ids = {
    # "Goedel-LM/Goedel-Prover-SFT",
    # "stoney0062/Leanabell-Prover-GD-RL",
    # "kaiyuy/leandojo-lean4-retriever-tacgen-byt5-small"
    # "stoney0062/Leanabell-Prover-DS-SFT",

    # "deepseek-ai/DeepSeek-Prover-V2-7B",
    # "deepseek-ai/DeepSeek-Prover-V1.5-RL",
    # "AI-MO/Kimina-Prover-Preview-Distill-7B",
    # "kfdong/STP_model_Lean",
    # "wellecks/llmstep-mathlib4-pythia2.8b"
    # "internlm/internlm2_5-step-prover",
    # "RickyDeSkywalker/TheoremLlama",
    # "ByteDance-Seed/BFS-Prover",
    # "ScalableMath/Lean-STaR-plus",
    # "deepseek-ai/DeepSeek-Prover-V1",
    "EleutherAI/llemma_34b", #optionally 7b as well
    "EleutherAI/llemma_7b"


}
batch_size = 1 # smaller batch size for demonstration

print("\n=====================================================================")
print(">>> Calculating Ensemble Diversity Coefficient (EDC)...")
print(f">>> Models in ensemble: {solver_model_ids}")
print(f">>> Batch Size: {batch_size}")
print("=====================================================================")

ensemble_diversity_coefficient = compute_edc(solver_model_ids, raw_ds, batch_size)

print("\n=====================================================================")
print(f"\nFinal Ensemble Diversity Coefficient (EDC): {ensemble_diversity_coefficient:.4f}")
print("=====================================================================")

>>> Loading and preparing dataset...


num_proc must be <= 5. Reducing num_proc to 5 for dataset of size 5.


Map (num_proc=5):   0%|          | 0/5 [00:00<?, ? examples/s]

>>> Dataset ready.
--- Using device: cuda ---
>>> Loading probe model...
>>> Probe model 'Saisam/gpt-neo-math-small' loaded with max length: 2048.

>>> Calculating Ensemble Diversity Coefficient (EDC)...
>>> Models in ensemble: {'EleutherAI/llemma_34b', 'EleutherAI/llemma_7b'}
>>> Batch Size: 1

>>> Computing cmdiv for pair: (EleutherAI/llemma_34b, EleutherAI/llemma_7b)
    ... processing batch 1/5


num_proc must be <= 1. Reducing num_proc to 1 for dataset of size 1.


...

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1 [00:00<?, ?it/s]


Initial loss 11.412302017211914 (step=0 epoch=0)

final loss step=0 epoch=0 of final layer loss 11.412302017211914 (note we are not recomputing loss after a step so this loss printed is larger than it should be/one off)
loss=11.412302017211914 (after fine tune, if not done it will be None)


Iteration:   0%|          | 0/1 [00:00<?, ?it/s]

dist between EleutherAI/llemma_34b and EleutherAI/llemma_7b: 0.9919756967830474
>>> Average distance (cmdiv) for pair = 0.0110
Diversity between EleutherAI/llemma_34b and EleutherAI/llemma_7b on json with batch size 1: 0.010988298315733913


Final Ensemble Diversity Coefficient (EDC): 0.0110
